In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

from PIL import Image, ImageFilter
import PIL
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle

In [2]:
from gen_pic_utils import *
import populate_library

## Select road type

In [7]:
road_type = 62 
car_type = 12

image_size = (1242,375)

In [8]:
def save(filename,data):
    with open(filename, 'w+') as f:
        pickle.dump(data, f)
        
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [9]:
# Init variables
poly_verts = 4
polygon = [(image_size[0],image_size[1]),(image_size[0],1),(1,1),(1,image_size[1])]
scales = [1, 1]
    
# Slider parameters
point_widget = FloatSlider(min=0, max=poly_verts-1, step=1, value=0)
x_widget = FloatSlider(min=1, max=image_size[0], step=1, value=image_size[0])
y_widget = FloatSlider(min=1, max=image_size[1], step=1, value=image_size[1])
car_widget = FloatSlider(min=0, max=1, step=1, value=0)
scale_widget = FloatSlider(min=0.1, max=2.0, step=0.1, value=1)
car_x_pos_widget = FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5)

def update_car_conf_widgets(*args):
    (x,y) = polygon[int(point_widget.value)]
    x_widget.value = x
    y_widget.value = y
    
def update_car_scale_widgets(*args):    
    scale = scales[int(car_widget.value)]
    scale_widget.value = scale
    
car_widget.observe(update_car_scale_widgets, 'value')    
point_widget.observe(update_car_conf_widgets, 'value')

def update_point_widgets(point,x,y,car_pos,scale,car_x_pos):
    
    point_id = int(point_widget.value)
    polygon[point_id] = (x_widget.value,y_widget.value)
    tmp_pt = list(polygon[3 - point_id])
    tmp_pt[1] = y_widget.value
    polygon[3 - point_id] = tuple(tmp_pt)    
        
    polygon[int(point_widget.value)] = (x_widget.value,y_widget.value)
    scales[int(car_pos)] = scale       
    
    lib = populate_library.populateLibrary()     
        
    road = lib.getElement("roads", road_type) 
    car = lib.getElement("cars", car_type)
    
    lrx = polygon[0][0]
    lry = polygon[0][1]
    llx = polygon[3][0]
    
    urx = polygon[1][0]
    ury = polygon[1][1]
    ulx = polygon[2][0]
    
    car.data = car.data.resize((int(car.data.size[0]*scale), int(car.data.size[1]*scale)), PIL.Image.ANTIALIAS)
    car_mask = car.data.getdata(3)
    mask = Image.new('L', car.data.size, color = 0)
    mask.putdata(car_mask)        
    
    if(car_pos == 0):
        car_x = int(llx + (lrx-llx)*car_x_pos - car.data.size[0]/2)
        car_y = int(lry - (car.data.size[1]/2))
    else:
        car_x = int(ulx + (urx-ulx)*car_x_pos - car.data.size[0]/2)
        car_y = int(ury - (car.data.size[1]/2))
    
    road.data.paste(car.data, (car_x,car_y), mask)    
    draw = ImageDraw.Draw(road.data)
    
    eps = 5
    draw.ellipse((x-eps, y-eps, x+eps, y+eps), fill = 'blue', outline ='blue')
    draw.polygon(polygon, fill=None, outline=(255,0,0,255))      
        
    display.display(road.data)
    

In [10]:
interact(update_point_widgets,point=point_widget, x=x_widget, y=y_widget, car_pos=car_widget, scale=scale_widget, car_x_pos=car_x_pos_widget);

In [20]:
(polygon,scales)

([(1242.0, 375.0), (1242, 1), (1, 1), (1, 375.0)], [1.0, 1])

In [11]:
configs = load('scene_configs')
configs[road_type] = (polygon,scales)
save('scene_configs',configs)

In [12]:
configs

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ([(786.0, 286.0), (651.0, 226.0), (602.0, 226.0), (495.0, 286.0)],
  [1.0, 0.2]),
 [],
 [],
 [],
 [],
 [],
 ([(835.0, 272.0), (698.0, 168.0), (563.0, 168.0), (441.0, 272.0)],
  [1.2, 0.6]),
 [],
 [],
 [],
 ([(757.0, 299.0), (660.0, 229.0), (592.0, 229.0), (466.0, 299.0)],
  [1.0, 0.3]),
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 